# Classification random forests

In [1]:
import numpy as np
import pandas as pd
import pickle
import random
import os

from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import IterativeStratification

In [2]:
os.chdir(r"C:/Users/mmandziej001/Desktop/Projects/FAIT/Prediction Module/POLAND_DANE/MODELING")

In [3]:
def bayesian_optimization(dataset, function, parameters):
    n_iterations = 10
    gp_params = {"alpha": 1e-4}

    BO = BayesianOptimization(function, parameters)
    BO.maximize(n_iter=n_iterations, **gp_params)

    return BO.max

In [4]:
def rfc_optimization(cv_splits):
    def function(n_estimators, max_depth, min_samples_split):
        return cross_val_score(
               RandomForestClassifier(
                   n_estimators=int(max(n_estimators,0)),                                                               
                   max_depth=int(max(max_depth,1)),
                   min_samples_split=int(max(min_samples_split,2)), 
                   n_jobs=-1, 
                   random_state=42,   
                   class_weight="balanced"),  
               X=test_data, 
               y=test_labels, 
               cv=10,
               scoring="roc_auc",
               n_jobs=-1).mean()

    parameters = {"n_estimators": (1, 1000),
                  "max_depth": (1, 2),
                  "min_samples_split": (5, 10)}
    return function, parameters

In [5]:
parameters = {"n_estimators": (1, 1000),
              "max_depth": (1, 2),
              "min_samples_split": (5, 10)}

In [6]:
def function(n_estimators, max_depth, min_samples_split):
        return cross_val_score(
               RandomForestClassifier(
                   n_estimators=int(max(n_estimators,0)),                                                               
                   max_depth=int(max(max_depth,1)),
                   min_samples_split=int(max(min_samples_split,2)), 
                   n_jobs=-1, 
                   random_state=42,   
                   class_weight="balanced"),  
               X=test_data, 
               y=test_labels, 
               cv=10,
               scoring='accuracy',
               n_jobs=-1).mean()

In [7]:
#Train model
def train(X_train, y_train, function, parameters):
    dataset = (X_train, y_train)
    cv_splits = 10
    
    best_solution = bayesian_optimization(dataset, function, parameters)      
    params = best_solution["params"]

    model = RandomForestClassifier(
             n_estimators = int(max(params["n_estimators"], 0)),
             max_depth = int(max(params["max_depth"], 1)),
             min_samples_split = int(max(params["min_samples_split"], 2)), 
             n_jobs = -1, 
             random_state = 42,   
             class_weight = "balanced")    
    model.fit(X_train, y_train)
    
    model_predict = model.predict(test_data)
    return model

def roc_auc_score_FIXED(y_true, y_pred):
    if len(np.unique(y_true)) == 1: # bug in roc_auc_score
        return accuracy_score(y_true, np.rint(y_pred))
    return roc_auc_score(y_true, y_pred)

In [8]:
train_data = pd.read_csv('model_training/data/data_train_under_inputed.csv')
test_data = pd.read_csv('model_training/data/data_test_under_inputed.csv')

train_id = train_data.pop('NIP')
test_id = test_data.pop('NIP')

train_labels = train_data.pop('Label')
test_labels = test_data.pop('Label')

train_datestamp = train_data.pop('DataUpadlosci')
test_datestamp = test_data.pop('DataUpadlosci')

In [16]:
boruta_importance = pd.read_excel('model_training/R_DUMPS/Boruta_variable_importance_under.xlsx')
boruta_importance = boruta_importance.sort_values(by=['normHits', 'meanImp'],
                                                  ascending=True)
boruta_features = list(
    boruta_importance[boruta_importance['decision'] == 'Confirmed'].V7)
boruta_pred_power = list(boruta_importance.V7)
boruta_features = [c for c in boruta_features if c not in [
    'DeclaredAccountsCount', 'RemovalDaysAgo',
    'X6', 'P4', 'RevenueToWages', 'RevenueToCash']]

In [25]:
#boruta_features = ['ScreenedParties', 'PartyType_Subsidiary', 'Critical_last_3_checklistsDR', 'ESR_Full_ESR_review', 'Minor_last_3_checklistsDR', 'Minor_last_3_checklistsPC', 'Major_last_3_checklistsPC', 'TLAssignedName_Yadav__N___Neha_', 'Cases_last_30_days_of_PC', 'Minor_last_10_checklistsDR', 'Critical_last_10_checklistsDR', 'Cases_last_5_days_of_PC', 'TLAssignedName_Makowska__M_M___Malgorzata_', 'TLAssignedName_Michalik__J___Justyna_', 'TeamExperience', 'HourNumeric', 'GroupCases', 'TLAssignedName_Jurojc__M___Mateusz_', 'ProcessingUnit_Gdansk', 'ProcessingUnit_MidCorp', 'FirstGroupCase', 'TLAssignedName_Jastrzebowska__S___Sonia_', 'Minor_last_10_checklistsPC', 'Major_last_10_checklistsPC', 'TLAssignedName_Marcos_Cantabrana__I___Ivan_', 'TLAssignedName_Wojciechowska__M___Magdalena_', 'ProjectExperience', 'Cases_last_30_days_of_DR', 'Cases_last_5_days_of_DR', 'TLAssignedName_Rybka__I_A___Izabela_Anna_', 'Major_last_10_checklistsDR', 'Major_last_3_checklistsDR', 'Major_last_5_checklistsDR']

In [17]:
test_data = test_data[boruta_features]  #[boruta_features] # used_columns, boruta_features
train_data = train_data[boruta_features]  #[boruta_features] # used_columns, boruta_features

In [18]:
print(list(train_data))
print(len(train_labels), sum(train_labels), round(sum(train_labels)/len(train_labels), 2))
print(len(test_labels), sum(test_labels), round(sum(test_labels)/len(test_labels), 2))

['EMISLegalForm_Other', 'ExpiredLicenses', 'ActiveLicenses', 'NoMail_NIE', 'FormaWlasnosci_WŁASNOŚĆ PRYWATNA KRAJOWA POZOSTAŁA', 'FormaWlasnosci_WŁASNOŚĆ ZAGRANICZNA', 'VirtualAccountsPresence_NIE', 'NoMail_TAK', 'PhoneNotPresent_NIE', 'PhoneNotPresent_TAK', 'FormaWlasnosci_Other', 'FormaWlasnosci_WŁASNOŚĆ KRAJOWYCH OSÓB FIZYCZNYCH', 'SpecialLegalForm_SPÓŁKI JAWNE', 'VirtualAccountsPresence_TAK', 'AdresBiuroWirtualne_NIE', 'RyzykownaDziałalnoscGlowna_NIE', 'AdresLokal_TAK', 'NoFax_TAK', 'NoFax_NIE', 'CAACEksport_NIE', 'NoWebsite_NIE', 'NoWebsite_TAK', 'AdresLokal_NIE', 'CAACImport_NIE', 'AffiliatesCount', 'DescriptionNull_NIE', 'AdresLokal_BrakDanych', 'DescriptionNull_TAK', 'MainPKD_BrakDanych', 'CAACImport_BrakDanych', 'CAACEksport_BrakDanych', 'RyzykowneDzialalnosciDodatkowe_BrakDanych', 'RyzykownaDziałalnoscGlowna_BrakDanych', 'AdresBiuroWirtualne_BrakDanych', 'NonCurrentLiabilities', 'ExternalIdsOthers', 'EMISLegalForm_PL-SK', 'RepresentationCount', 'RetainedEarnings', 'CashandCas

In [19]:
best_model_rfc = train(train_data, train_labels, function, parameters)

|   iter    |  target   | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        |  0.7672   |  1.901    |  9.914    |  458.6    |
|  2        |  0.759    |  1.885    |  6.502    |  404.0    |
|  3        |  0.762    |  1.602    |  5.028    |  579.6    |
|  4        |  0.7678   |  1.584    |  6.607    |  785.9    |
|  5        |  0.7705   |  1.908    |  6.665    |  85.31    |
|  6        |  0.7705   |  1.491    |  7.44     |  85.92    |
|  7        |  0.8007   |  1.086    |  9.884    |  32.0     |
|  8        |  0.8149   |  1.012    |  9.384    |  5.856    |
|  9        |  0.7709   |  1.936    |  6.178    |  999.9    |
|  10       |  0.5965   |  1.0      |  5.0      |  1.0      |
|  11       |  0.798    |  1.019    |  10.0     |  8.993    |
|  12       |  0.7951   |  1.178    |  5.813    |  34.03    |
|  13       |  0.8097   |  1.426    |  6.265    |  28.96    |
|  14       |  0.8174   |  1.713    |  9.672    |  25.33    |
|  15   

In [20]:
model = best_model_rfc
#model = RandomForestClassifier(max_depth=2, random_state=5, n_estimators=10000, class_weight="balanced", max_features=1)

model.fit(train_data, train_labels)
model_predict = model.predict(test_data)
test_predictions = model.predict_proba(test_data)[:, 1]
train_predictions = model.predict_proba(train_data)[:,1]
auc_best_model_train = roc_auc_score(train_labels, train_predictions)
auc_best_model_test = roc_auc_score(test_labels, test_predictions)
overfit = round((auc_best_model_train - auc_best_model_test), 3)

print(auc_best_model_train, auc_best_model_test, overfit)

0.8969840882919059 0.8954327114359251 0.002


In [21]:
test_predictions_df = pd.DataFrame(test_predictions)
train_predictions_df = pd.DataFrame(train_predictions)

test_results = pd.concat([test_id, test_datestamp, test_predictions_df, test_labels], axis=1)
test_results = test_results.rename(columns={0: "Score"})
test_results = test_results.sort_values(by='Score', axis=0, ascending=False)

SumTest = pd.DataFrame(np.cumsum(test_results['Label']))
SumTest = SumTest.rename(columns={'Label': 'Sum'})
test_results = pd.concat([test_results, SumTest], axis=1)
test_results.reset_index(inplace = True , drop=True)
X_coordinates = pd.DataFrame(np.arange(1,(test_results.shape[0]+1))/(test_results.shape[0]))
X_coordinates = X_coordinates.rename(columns={0: 'X_coordinates'})
y_coordinates = pd.DataFrame(test_results['Sum']/test_results['Sum'].max())
y_coordinates = y_coordinates.rename(columns={'Sum': 'y_coordinates'})
test_results = pd.concat([test_results, X_coordinates,y_coordinates], axis=1)

# train results
train_results = pd.concat([train_id, train_datestamp, train_predictions_df, train_labels], axis=1)
train_results = train_results.rename(columns={0:"Score"})
train_results = train_results.sort_values(by='Score', axis=0, ascending=False)

SumTrain = pd.DataFrame(np.cumsum(train_results['Label']))
SumTrain = SumTrain.rename(columns={'Label':'Sum'})
train_results = pd.concat([train_results, SumTrain], axis=1)
train_results.reset_index(inplace = True , drop=True)

X_coordinates = pd.DataFrame(np.arange(1,(train_results.shape[0]+1))/(train_results.shape[0]))
X_coordinates = X_coordinates.rename(columns={0: 'X_coordinates'})
y_coordinates = pd.DataFrame(train_results['Sum']/train_results['Sum'].max())
y_coordinates = y_coordinates.rename(columns={'Sum': 'y_coordinates'})
train_results = pd.concat([train_results, X_coordinates,y_coordinates], axis=1)

In [22]:
train_results.to_csv(r'model_training/results/rf/rf_train_results.csv')
test_results.to_csv(r'model_training/results/rf/rf_test_results.csv')
filename = r'model_training/results/rf/rfc_model.sav'
pickle.dump(model, open(filename, 'wb'))
with open('model_training/results/rf/used_features.txt', 'w') as f:
    for item in train_data.columns:
        f.write("%s\n" % item)
print(list(train_data.columns))

['EMISLegalForm_Other', 'ExpiredLicenses', 'ActiveLicenses', 'NoMail_NIE', 'FormaWlasnosci_WŁASNOŚĆ PRYWATNA KRAJOWA POZOSTAŁA', 'FormaWlasnosci_WŁASNOŚĆ ZAGRANICZNA', 'VirtualAccountsPresence_NIE', 'NoMail_TAK', 'PhoneNotPresent_NIE', 'PhoneNotPresent_TAK', 'FormaWlasnosci_Other', 'FormaWlasnosci_WŁASNOŚĆ KRAJOWYCH OSÓB FIZYCZNYCH', 'SpecialLegalForm_SPÓŁKI JAWNE', 'VirtualAccountsPresence_TAK', 'AdresBiuroWirtualne_NIE', 'RyzykownaDziałalnoscGlowna_NIE', 'AdresLokal_TAK', 'NoFax_TAK', 'NoFax_NIE', 'CAACEksport_NIE', 'NoWebsite_NIE', 'NoWebsite_TAK', 'AdresLokal_NIE', 'CAACImport_NIE', 'AffiliatesCount', 'DescriptionNull_NIE', 'AdresLokal_BrakDanych', 'DescriptionNull_TAK', 'MainPKD_BrakDanych', 'CAACImport_BrakDanych', 'CAACEksport_BrakDanych', 'RyzykowneDzialalnosciDodatkowe_BrakDanych', 'RyzykownaDziałalnoscGlowna_BrakDanych', 'AdresBiuroWirtualne_BrakDanych', 'NonCurrentLiabilities', 'ExternalIdsOthers', 'EMISLegalForm_PL-SK', 'RepresentationCount', 'RetainedEarnings', 'CashandCas